<h1><center>Temporal evolution of species-collectors networks</center></h1>
<h2><center>University of Brasília Herbarium (UB) dataset</center></h2>

<h4><center>Author: Pedro Correia de Siracusa</center></h4>
<h4><center>Date: May 23, 2017</center></h4>

---

On section 3 of the previous notebook I explored the structure of a species-collector network at Fazenda Água Limpa and identified the top-5 collectors in terms of the number of species they have collected. Now I will take a closer look at the recording history of those collectors. Here I explore the temporal evolution of the network formed by collectors and the species they have recorded (species-collectors network).

<h3>Table of Contents</h3>

<ul>
    <li><a href="#1">1. Collectors and their records through time</a>
        <ul>
            <li><a href="#1.1">1.1 The overlapped view</a></li>
            <li><a href="#1.2">1.2 The snapshot view</a></li>
        </ul>     
    </li>
    
    <li><a href="#2">2. Who is Ratter, JA?</a>
    </li>
    
    <li><a href="#3">3. Who has worked with whom?</a>
        <ul>
            <li><a href="#3.1">3.1 The species-collectors network</a></li>
            <li><a href="#3.2">3.2 Understanding the story</a></li>
        </ul>    
    </li>
    
</ul>        

In [1]:
import os,datetime, subprocess

import numpy as np
import pandas as pd
import matplotlib as mpl
import networkx as nx

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize']=(10,10)

animations_dir = os.path.join(os.path.curdir, 'animations')

In [2]:
colsList = ['scientificName', 'taxonRank', 'family', 
            'stateProvince', 'locality', 'municipality', 
            'hasCoordinate', 'decimalLatitude', 'decimalLongitude',
            'recordedBy', 'identifiedBy', 'occurrenceRemarks',
            'eventDate', 'issue']

In [3]:
ub_dset = pd.read_csv('./0077202-160910150852091/occurrence.txt', sep='\t', 
                 warn_bad_lines=False, 
                 error_bad_lines=False, 
                 usecols=colsList)

/home/pedro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (217) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


For this analysis I will only use records which refer to the taxonomic resolution of `SPECIES`. Also, I will filter out records with null values for either the name of the collector (`recordedBy`) or the collection date (`eventDate`). As the records at the `eventDate` columns are retrieved from the data file as strings, I will explicitly convert them to `datetime` type.

In [4]:
# filter data
ub_dset = ub_dset[ ub_dset['recordedBy'].notnull() & ub_dset['eventDate'].notnull() ]
ub_dset = ub_dset[ ub_dset['taxonRank']=='SPECIES' ]

# convert eventDate strings to date
ub_dset['eventDate'] = pd.to_datetime(ub_dset['eventDate'], format="%Y-%m-%dT%H:%MZ")

In [5]:
ub_dset.head()

,recordedBy,occurrenceRemarks,eventDate,stateProvince,municipality,locality,decimalLatitude,decimalLongitude,identifiedBy,scientificName,family,taxonRank,issue,hasCoordinate
0,"Anderson, WR; Arroyo, MTK; Hill, SR; Santos, R...","Rocky slope.; Shrub 0,5 m. Fruto green, mature...",1973-03-11 01:00:00,Goiás,NaN,8 km by road W of Monte Alegre.,0.00,0.00,"Delprete, PG",Sabicea brasiliensis Wernham,Rubiaceae,SPECIES,ZERO_COORDINATE;GEODETIC_DATUM_ASSUMED_WGS84;C...,True
1,"Irwin, HS; Souza, R; Santos, RR","Corolla white.; Shrub 1 m. Fruto green, maturi...",1965-12-03 01:00:00,Goiás,NaN,Serra de Pirineus. 20 km S. of Corumbá de Goiás.,-16.00,-49.00,"Delprete, PG",Sabicea brasiliensis Wernham,Rubiaceae,SPECIES,GEODETIC_DATUM_ASSUMED_WGS84,True
2,"Heringer, EP; Eiten, G",Soil full of quartz stones derived from quartz...,1974-12-20 01:00:00,Goiás,Caldas Novas,"At headwaters of creek, Rio Quente, near hotel...",-17.80,-48.75,"Delprete, PG",Sabicea brasiliensis Wernham,Rubiaceae,SPECIES,GEODETIC_DATUM_ASSUMED_WGS84,True
3,"Anderson, WR; Hill, SR; Santos, RR; Souza, R","Shrub 0,4 m. Fruto red.; Cerrado.",1973-05-11 01:00:00,Goiás,NaN,Serra Dourada. 16 km S of Goiás Velho.,0.00,0.00,"Delprete, PG",Sabicea brasiliensis Wernham,Rubiaceae,SPECIES,ZERO_COORDINATE;GEODETIC_DATUM_ASSUMED_WGS84;C...,True
4,"Harley, RM; Giulietti, AML",Small tree 3 m; Cerrado.,1968-12-26 01:00:00,Goiás,Pirenópolis,NaN,-15.75,-48.75,"Delprete, PG",Rudgea viburnoides (Cham.) Benth.,Rubiaceae,SPECIES,GEODETIC_DATUM_ASSUMED_WGS84,True


# 1. Collectors and their records through time
<a name="1"></a>

On this section I develop the first routines for obtaining species-collectors networks that evolve in time. First I build a timeseries of all the records obtained by the collectors of our interest, which will be used to build a graph containing all the collectors and species they have recorded. Given a period of interest (monthly or yearly), the occurrences will be aggregated into frames, which will be displayed sequentially and compose a simple animation.

First we must define a list of collectors of interest, which will initially be used to filter the dataset. If there is only one collector of interest, it should be a list of length 1.

In [6]:
collectors = ['Ratter, JA']

By inspecting the `recordedBy` column of the dataset, however, we realize that it is not atomic. The names of the collectors which are responsible for each record are separated by semicolons. The following routine retrieves all the records in the dataset which have been recorded by the collectors passed in:

In [7]:
def getRecordsBy(df, collectors):
    df = df[df['recordedBy'].notnull()]
    if df.size>0:
        col_lists = df['recordedBy'].apply( lambda row: [ c.strip() for c in row.split(';') if c.strip()!='']) 
        return df[ [ any(c in collectors for c in col_list) for col_list in col_lists ] ]
    else:
        return df

Then, in order to deal with timeseries operations on the dataset optimally I will index it on the `eventDate` column. For more details on time indexing please refer to the pandas timeseries [reference](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#indexing). The timeseries will be assigned to the variable `ts`.

In [8]:
ts = getRecordsBy(ub_dset, collectors).sort_values('eventDate').set_index('eventDate', drop=True)

The graph is then built using the entire timeseries. Note that the graph's layout (coordinates of each node, for visualization) will be calculated once based on all its nodes, and assigned to the variable `pos`. This early positioning guarantees that each node will not change its coordinates across each frame on the animation. 

In [9]:
G = nx.Graph()
G.add_nodes_from(collectors, bipartite=0)
G.add_nodes_from(ts['scientificName'], bipartite=1)
G.add_edges_from([ (col,sp) for col in collectors for sp in getRecordsBy(ts,col)['scientificName'] ])

pos = nx.spring_layout(G)

The frames set will be represented as a dictionary in which each individual frame (also represented by a `dict`) is indexed by its number. Each frame contains information that is relevant for selectively displaying nodes and edges. For building each frame the routine below converts the datetime index into a period index, which represent, in pandas, a [time interval](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-span-representation). The frequency of the period is passed in as the argument `periodFreq` as a [string alias](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases).

In [10]:
def buildFramesDict(ts, collectors, periodFreq='A'):
    
    frames = {}
    ts = getRecordsBy(ts, collectors)
    
    periodRange = pd.period_range(start=ts.index[0], end=ts.index[-1], freq=periodFreq)
    ts = ts.to_period(periodFreq)

    for i,p in enumerate(periodRange):
        ts_thisperiod = ts[str(p)]
        ts_uptothisperiod = ts.loc[periodRange[0:i+1]]
        
        splist = list(ts_thisperiod['scientificName'])
        cum_sp_counts = ts_uptothisperiod['scientificName'].value_counts()
        
        frames[i]={}
        frames[i]['period'] = p
        frames[i]['collectors'] = collectors
        frames[i]['species'] = splist
        frames[i]['species_counts'] = [ cum_sp_counts[sp] for sp in splist ]
        frames[i]['edges'] = [ (col,sp) for col in collectors for sp in getRecordsBy(ts_thisperiod, col)['scientificName'] ]
        frames[i]['collectors_k'] = [ getRecordsBy(ts_uptothisperiod, c)['scientificName'].unique().size for c in collectors ]
               
    return frames

Let's get the frames for our graph, built from the history of records from *Ratter, JA*: 

In [11]:
frames_dict = buildFramesDict(ts, collectors, periodFreq='A')

To get an overview of the structure of each frame we take a look at frame 0. The key `collectors_k` refers to the cumulative degree of each collector at the specific frame. Being cumulative means that the degree is increased by the number of new species collected at new frames. The key `species_counts` is the cumulative count (since the first frame) for each species on a given frame.

In [12]:
frames_dict[0]

{'collectors': ['Ratter, JA'],
 'collectors_k': [4],
 'edges': [('Ratter, JA', 'Rapanea guianensis Aubl.'),
  ('Ratter, JA', 'Eugenia aurata O.Berg'),
  ('Ratter, JA', 'Ocotea minarum (Nees & Mart.) Mez'),
  ('Ratter, JA', 'Pradosia brevipes (Pierre) T.D.Penn.')],
 'period': Period('1966', 'A-DEC'),
 'species': ['Rapanea guianensis Aubl.',
  'Eugenia aurata O.Berg',
  'Ocotea minarum (Nees & Mart.) Mez',
  'Pradosia brevipes (Pierre) T.D.Penn.'],
 'species_counts': [1, 1, 1, 1]}

Now we can render the frames individually and then use *ffmpeg* to create the animations. The network evolution in time can be inspected in two distinct views. The first is the **overlapped view**, which adds new nodes to previous plots in a cumulative fashion. The second is the **snapshot view**, which plots each frame individually, clearing the plot for consecutive frames.

In [13]:
def renderFrame(G, pos, frames_dict, frameNum, scalingFactor=20, overlapped=True):
    
    speciesAtFrame = frames_dict[frameNum]['species']
    speciesAtFrame_sizes = [i*scalingFactor for i in frames_dict[frameNum]['species_counts'] ]
    edgesAtFrame = frames_dict[frameNum]['edges']

    # draw species nodes
    nx.draw_networkx_nodes(G, pos=pos, 
                           nodelist=speciesAtFrame, 
                           node_size=speciesAtFrame_sizes, alpha=0.4)

    # draw collector nodes
    nx.draw_networkx_nodes(G, pos=pos, 
                           nodelist=frames_dict[frameNum]['collectors'], node_color='b', 
                           node_size=frames_dict[frameNum]['collectors_k'] )
    
    # draw edges
    nx.draw_networkx_edges(G, pos=pos, edgelist=edgesAtFrame, width=0.3, alpha=0.3 )
    
    
    if (overlapped==True and frameNum==0) or overlapped==False:
        nx.draw_networkx_labels(G, pos=pos, labels=dict([ (c,c) for c in frames_dict[frameNum]['collectors']]) )
    
    plt.title('Period: '+str(frames_dict[frameNum]['period']))  
    return
    

def renderFrames(saveDir, overlapped=True, **kwargs):
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
        
    for i in frames_dict.keys():
        renderFrame(G, pos, frames_dict, frameNum=i, overlapped=overlapped)
        
        try: plt.xlim(kwargs['xlim'])
        except: plt.xlim([-0.1, 1.1])
   
        try: plt.ylim(kwargs['ylim'])
        except: plt.ylim([-0.1, 1.1])
            
        plt.savefig(os.path.join(saveDir,'fr'+str(i)+'.png'))
        
        if not overlapped: 
            plt.clf()
            
    return plt.clf()


def buildAnimation(animName, overlapped=True, **kwargs):
    save_dir = os.path.join(animations_dir, animName)
    renderFrames( save_dir, overlapped=overlapped, **kwargs )
    
    # renders video from images
    cmd = "ffmpeg -r 2 -s 1024x720 -start_number 0 -i fr%d.png -vcodec libx264 ../"+animName+".mp4"
    subprocess.Popen(cmd, cwd=save_dir, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

## 1.1 The overlapped view
<a name="1.1"></a>

This animation shows the temporal evolution of the species-collector network for the collector *Ratter, JA*. Each red node represents a particular species, and the number of times it was recorded by *Ratter* is represented by its size. The blue node represents *Ratter* himself, which grows according to the number of different species he has collected so far. The overlapped view allows us to have an idea of the importance of the collector in a given time, as we plot species nodes cumulatively.

In [14]:
buildAnimation('n2_anim1_overlapped')

In [15]:
%%HTML
<video width="720" height="720" controls autoplay loop>
  <source src="animations/n2_anim1_overlapped.mp4" type="video/mp4">
</video>

## 1.2 The snapshot view
<a name="1.2"></a>

This animation shows a snapshot view of the temporal evolution of the species-collector network for the collector *Ratter, JA*. This gives us a better idea of how productive the collector was at each period individually. Although we do not have anymore the information of which species have been collected up to a given period, we still have some idea of the importance of the collector on the network by inspecting his node's size, which represents his cumulative degree. From the animation note that *Ratter* has recorded many species particularly on the years of 1968 and 1976 whereas there was apparently no field work on 1970, 1988 or 1995. One limitation of this view is that it is hard to track a particular species in terms of the temporal distribution of its recordings. The total number of times it was recorded by the collectors of interest since the beginning, though, is reflected on the size of the node which represents it. 

In [16]:
buildAnimation('n2_anim1_snapshot', overlapped=False)

In [17]:
%%HTML
<video width="720" height="720" controls autoplay loop>
  <source src="animations/n2_anim1_snapshot.mp4" type="video/mp4">
</video>

# 2. Who is *Ratter, JA*?
<a name="2"></a>

After obtaining the animations above for *Ratter, JA*, "who is *Ratter*?", we might ask. *James Ratter* happens to be a collector of the [*Royal Botanic Garden of Edinburgh*](http://www.rbge.org.uk/science/herbarium/about-the-collections/collectors#J_ratter). Since 1966 he has been interested on the Cerrado biome and has been involved with the University of Brasilia, being appointed as one of the founders of the Ecology Department at that university. He was an assiduous visitor, and has contributed ever since to the UnB herbarium collection. From the dataset, apparently his last visit to Brazil was in 2009.

In [18]:
ts.tail(4)

,recordedBy,occurrenceRemarks,stateProvince,municipality,locality,decimalLatitude,decimalLongitude,identifiedBy,scientificName,family,taxonRank,issue,hasCoordinate
eventDate,,,,,,,,,,,,,
2000-08-09 02:00:00,"Ratter, JA; Bridgewater, S; Fonsêca Filho, J; ...","Small tree , 4m tall, 10 cm dbh, with fawnish ...",Bahia,Cocos,Fazenda Trijunção,-14.816667,-45.966667,Incógnito,Neea theifera Oerst.,Nyctaginaceae,SPECIES,GEODETIC_DATUM_ASSUMED_WGS84,True
2003-10-18 02:00:00,"Ratter, JA","Bambu em touceira formando mancha 1,2m; Cerrado.",Minas Gerais,Formoso,Em uma encosta na margem da Fazenda Trijunção ...,-14.833333,-46.000000,"Filgueiras, TS",Filgueirasia arenicola (McClure) Guala,Poaceae,SPECIES,GEODETIC_DATUM_ASSUMED_WGS84,True
2003-11-13 01:00:00,"Ratter, JA",.,Distrito Federal,Planaltina,Águas Emendadas,-15.533333,-47.550000,"Proença, CEB",Erythroxylum engleri O. E. Schulz,Erythroxylaceae,SPECIES,GEODETIC_DATUM_ASSUMED_WGS84,True
2009-05-04 02:00:00,"Ratter, JA",NaN,Mato Grosso,Vale dos Sonhos,NaN,0.000000,0.000000,"Bigio, NC",Pera barbinervis Pax & K.Hoffm.,Peraceae,SPECIES,ZERO_COORDINATE;GEODETIC_DATUM_ASSUMED_WGS84;C...,True


It is odd, though, that he has managed to travel to Brazil only for collecting a single species once. This data seems to be inconsistent. It might be the case that the recording date is wrong for this occurrence; or that *Ratter* was not in fact the collector. Or that might even be the case that although *Ratter* was present at the field trip, he was not activelly collecting specimens. Perhaps we might try to figure out such questions by looking at the network of researches who have done field work with him.

# 3. Who has worked with whom?
<a name="3"></a>

Here I explore the possibility of inferring scientific interactions between collectors based on the temporal analysis of the species-collectors network. This could be useful for example if we want to systematically uncover some underlying aspects of the species collectors community. Some interesting questions emerge: 

* What is the role of a given collector in the scientific community? Is he/she a professor? a student? a lab assistant? 
* Is a collector currently retired of field work? Was he/she a pioneer on a particular species group?
* Does a collector hold a scientific relationship with other important collectors (e.g. advisor/advised by)? 
* Does a collector work with interdisciplinary teams (in terms of taxonomic groups)? Or they almost always perform field work with specialists on their own interest group? 
* In which context data was collected? Was it on a field class? On a survey for sampling a specific group of organism? On a generalist survey?

Such aspects might be relevant to be considered if we were to characterize sampling bias on the occurrence data of a species or group; or if we were to identify regions that are assessible for field work and could be lacking data for a taxonomic group. 

## 3.1 The species-collectors network
<a name="3.1"></a>

On the previous notebook I have identified a particular group of three collectors that shared a fair number of species. They were:

In [19]:
collectors = ['Munhoz, CBR', 'Eugênio, CUO', 'Amaral, AG']

Let's get their records and build the species-collector network. Then we inspect both the overlapped and the snapshot view of the time-varying network.

In [20]:
ts = getRecordsBy(ub_dset, collectors).sort_values('eventDate').set_index('eventDate', drop=True)

In [21]:
G = nx.Graph()
G.add_nodes_from(collectors, bipartite=0)
G.add_nodes_from(ts['scientificName'], bipartite=1)
G.add_edges_from([ (col,sp) for col in collectors for sp in getRecordsBy(ts,col)['scientificName'] ])

pos = nx.spring_layout(G)

In [22]:
frames_dict = buildFramesDict(ts, collectors, periodFreq='A')

In [23]:
buildAnimation('n2_anim2_overlapped')
buildAnimation('n2_anim2_snapshot', overlapped=False)

In [24]:
%%HTML
<h3>Overlapped view</h3>
<video width="720" height="720" controls autoplay loop>
  <source src="animations/n2_anim2_overlapped.mp4" type="video/mp4">
</video>

<hr/>
<h3>Snapshot view</h3>
<video width="720" height="720" controls autoplay loop>
  <source src="animations/n2_anim2_snapshot.mp4" type="video/mp4">
</video>

The animations above suggest that *Munhoz, CBR* started effectively recording species around 1992. From the snapshot view we can try to identify her most productive years in terms of species recordings as well as those years where no field work was done. The period from 1992 to 2000 seems to have two 'peaks' in terms of fieldwork effort: the first is around 1994 and the second 1997/98. From 2000 to 2005 most fieldwork is concentrated on the year of 2003. No fieldwork at all seems to having been done in 2002 and 2005.

*Amaral* and *Eugênio* have both started recording in 2006. From 2006 to 2008 *Eugênio* has done a lot of fieldwork, and many of the species recorded by him were also recorded by *Amaral* and *Munhoz* on the same years. *Amaral* has done almost no fieldwork from 2008 to 2011, though. A high number of shared species between two collectors on a given time period relatively to the number of records they have obtained alone may be an indicative that they have done fieldwork together. If we then refined the frequency of the period (say, from yearly to monthly) and still observed a high number of shared species on each frame that would increase our confidence on the hypothesis that they have worked together. So the animation suggests that *Munhoz*, *Eugênio* and *Amaral* have all worked together in 2006 and 2007. *Munhoz* and *Eugênio* (without *Amaral*) have worked together in 2008.

The years from 2010 to 2015 were not as productive for *Eugênio*. On the other hand, *Amaral* has increased her sampling effort from 2012 to 2015. In 2012 all except one species collected by *Munhoz* were also collected by *Amaral*. On the other hand *Amaral* has collected many species that were never collected by *Munhoz* (check the overlapped view in 2012). That suggests that although *Munhoz* might have done most of her 2012 fieldwork together with *Amaral*, *Amaral* has done a lot of fieldwork alone (or with other collectors not considered here). Then from 2013 to 2015 practically all species recorded by *Amaral* were also recorded by *Munhoz*.

Why have these three researchers put so much effort on the same species during small periods of time (2-3 years)? Next I will address this question by looking at the research history of these three collectors and try to find possible explanations for the observed patterns in the data.

## 3.2 Understanding the story
<a name="3.2"></a>

On the last section we explored temporal aspects of the species-collectors network for collectors *Munhoz*, *Eugênio* and *Amaral*. Now let's try to see the big picture. I will use the [lattes platform](http://lattes.cnpq.br/) to search for for the research profile and history of each collector.

[*Cássia Beatriz Munhoz (Munhoz, CBR)*](http://buscatextual.cnpq.br/buscatextual/visualizacv.do?id=K4791009T9) is a professor at the University of Brasilia and a curator of the university herbarium. She is mostly interested on herbaceous plants (as it was suggested on the previous notebook). She started her undergraduate studies in Biology at the University of Brasilia in 1989 and got her degree in 1992. This is consistent with our analysis above, in that she might have started a more intense fieldwork around her last undergraduate semesters (or even after obtaining her degree). She was a curator at the Brasília botanical garden (JBB) from 1993 to 1994, which might have contributed for the high number of species recorded on that period. From 1995 to 1996 she was a Master's candidate (would the lower number of species recorded on 1996 be related to her commitment on analysing her data and writing her dissertation?). From 1997 to 1998 she went again to work at the Brasília botanical garden, and also got a grant from FAPDF for collecting botanical material (1998/99). From 1999 to 2003 she was a Doctoral candidate. She hasn't done much field work from 1999 to 2002. The year of 2003 was noticeably productive, although I could not find out a particular reason for that. 

[Chesterton Ulysses Orlando Eugênio (Eugênio, CUO)](http://buscatextual.cnpq.br/buscatextual/visualizacv.do?id=K4262136P6) started his undergraduate in Biology in 2004 and obtained his  degree in 2009, with a monograph on temporal dynamics of herbaceous plants. His work was advised by *Munhoz*. From 2006 to 2008 he did an internship at his university (UCB) on structural aspects of herbaceous communities at Fazenda Água Limpa. Although the author does not mention it in his profile this work was probably advised by *Munhoz* because of her expertise on the group and because she was his advisor on his monograph. The data collected during this period was probably used for his monograph. Note also that he has done no field work in 2009, which might again reflect his commitment on writing his monograph. In 2010 he participated of a floristic assessment with the ComCerrado network, which explains the high number of records for that year. *Eugênio* has published several papers with *Amaral*, which indicates that they may have done field work together in fact (or at least were interested on the same species).

[Aryanne Gonçalves Amaral (Amaral, AG)](http://buscatextual.cnpq.br/buscatextual/visualizacv.do?id=K4750617A7) has her first records in 2006, which was when she obtained her undergraduate degree in Biology from UCB. Her monograph, also advised by *Munhoz*, was a case study on management aspects of the public park *Parque Águas Claras*. Her first records were probably obtained during her Master's degree (2006-2008) though, as the network suggests intensified field work in 2006 and 2007. Similarly to *Eugênio*, she also studied structural aspects on the communities of herbaceous plants at Fazenda Água Limpa. There was almost no field work in 2009, which might be a consequence of her commitment on writing her dissertation. She was a Doctoral candidate from 2011 to 2015. The title and description she gave for her project on her profile suggests that there was in fact intense field work during her program. Although her program started in 2011, she started collecting field data in 2012. *Munhoz* was her advisor during her Doctoral program and co-advisor during her Master program (*Jeanine Maria* was her advisor). Together with *Munhoz* and *Eugênio*, *Amaral* is has been an integrant of a research project named *Cerrados do Planalto Central: estrutura, dinâmica e processos ecológicos* since 2010.